In [3]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd


# Import Dataset, Processing

In [67]:
DF = pd.read_csv('gurobiTime.csv',index_col=0)
DF = DF.drop(['player.1'],axis=1)


positionDF = []
num_positions = len(DF['position'].unique())


for i in DF['position'].unique():
    positionDF.append( (DF[DF['position'] == i]) )

    

[]

Make Gurobi datastructures to help represent the problem

We create dictionaries for every single variable for each player, to allow for efficient access and ease of use.  The keys are Players, the tupleList objects are returnPlayer, playerVariance, playerCost , and position


QB, RB, RB, WR, WR, WR, TE, D, K 



In [96]:
bigDict = {}
for df_iter in range(len(positionDF)):
    
    currPosition = positionDF[df_iter]  
    for player in currPosition.index:
        returnPlayer = currPosition.loc[player]['mean']
        playerVariance = currPosition.loc[player]['variance']
        playerCost = currPosition.loc[player]['cost']
        position = currPosition.loc[player]['position']
        bigDict[player] = [returnPlayer, playerVariance, playerCost , position]
         
player, returnPlayer, playerVariance, playerCost , position = gp.multidict(bigDict)


positionConstraints = gp.tupledict( { 'QB':1, "RB":2,"WR":3, 'TE':1,"D":1,'K':1} )



[1]

In [89]:
#

180.292

In [ ]:
try:
    m = gp.Model('QMIP')
    #add binary decision variables on whether 
    for i in DF['position'].unique():
        numPlayers = len(DF[DF['position']==i])
        varName = 'player' + i
        x = m.addVar( numPlayers , vtype=GRB.BINARY, name = varName  )
        listOfPositionDecisionVars.append(x)
    
    
    # set up return values for each player
    for df_iter in range(len(positionDF)):
        currentDF = positionDF[df_iter]
        currentPlayers = listOfPositionDecisionVars[df_iter] 
        for player in currentPlayers:
            currentDF['mean']
        
    
    
    m.update()

except gp.GurobiError as e:
    print('Error code ' + str(e.errno) + ": " + str(e))